In [1]:
%pip install --upgrade pip

# Uninstall conflicting packages
%pip uninstall -y langchain-core langchain-openai langchain-experimental langchain-community langchain chromadb beautifulsoup4 python-dotenv PyPDF2 rank_bm25 faiss-cpu weaviate-client langchain-weaviate

# Install compatible versions of langchain-core and langchain-openai
%pip install langchain-core==0.3.6
%pip install langchain-openai==0.2.1
%pip install langchain-experimental==0.3.2
%pip install langchain-community==0.3.1
%pip install langchain==0.3.1

# Install remaining packages
%pip install chromadb==0.5.11
%pip install python-dotenv==1.0.1
%pip install PyPDF2==3.0.1 -q --user
%pip install rank_bm25==0.2.2

# new vector stores
%pip install faiss-cpu==1.8.0.post1
%pip install weaviate-client==4.8.1
%pip install langchain-weaviate==0.0.3

Note: you may need to restart the kernel to use updated packages.
Found existing installation: langchain-core 0.2.43
Uninstalling langchain-core-0.2.43:
  Successfully uninstalled langchain-core-0.2.43
Found existing installation: langchain-openai 0.1.25
Uninstalling langchain-openai-0.1.25:
  Successfully uninstalled langchain-openai-0.1.25
Found existing installation: langchain-experimental 0.3.2
Uninstalling langchain-experimental-0.3.2:
  Successfully uninstalled langchain-experimental-0.3.2
Found existing installation: langchain-community 0.2.19
Uninstalling langchain-community-0.2.19:
  Successfully uninstalled langchain-community-0.2.19
Found existing installation: langchain 0.2.17
Uninstalling langchain-0.2.17:
  Successfully uninstalled langchain-0.2.17
Found existing installation: chromadb 0.5.11
Uninstalling chromadb-0.5.11:
  Successfully uninstalled chromadb-0.5.11
Found existing installation: python-dotenv 1.0.1
Uninstalling python-dotenv-1.0.1:
  Successfully uninstalled

In [2]:
import os
os.environ['USER_AGENT'] = 'RAGUserAgent'
import openai
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
import chromadb
from langchain_community.vectorstores import Chroma
from langchain_core.runnables import RunnableParallel
from dotenv import load_dotenv, find_dotenv
from langchain_core.prompts import PromptTemplate
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents.base import Document
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever

In [3]:
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
openai.api_key = os.environ['OPENAI_API_KEY']
embedding_function = OpenAIEmbeddings()
llm = ChatOpenAI(model_name="gpt-4o-mini")
pdf_path = "google-2023-environmental-report.pdf"
collection_name = "google_environmental_report"
str_output_parser = StrOutputParser()
user_query = "What are Google's environmental initiatives?"

In [4]:
pdf_reader = PdfReader(pdf_path)
text = ""
for page in pdf_reader.pages:
    text += page.extract_text()

In [5]:
character_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ". ", " ", ""],
    chunk_size=1000,
    chunk_overlap=200
)
splits = character_splitter.split_text(text)

In [6]:
dense_documents = [Document(page_content=text, metadata={"id": str(i), "source": "dense"}) for i, text in enumerate(splits)]
sparse_documents = [Document(page_content=text, metadata={"id": str(i), "source": "sparse"}) for i, text in enumerate(splits)]


In [7]:
chroma_client = chromadb.Client()
vectorstore = Chroma.from_documents(
    documents=dense_documents,
    embedding=embedding_function,
    collection_name=collection_name,
    client=chroma_client
)

In [8]:
from langchain_community.vectorstores import FAISS
vectorstore = FAISS.from_documents(
    documents=dense_documents,
    embedding=embedding_function
)


In [19]:
import weaviate
from langchain_weaviate.vectorstores import WeaviateVectorStore
from weaviate.embedded import EmbeddedOptions
from langchain.vectorstores import Weaviate
from langchain.schema import Document
from tqdm import tqdm
import ssl
import os
import urllib.request

# Most direct way to disable SSL verification for development
ssl._create_default_https_context = ssl._create_unverified_context

collection_name = "Google_environmental_report"

client = weaviate.WeaviateClient(
    embedded_options=EmbeddedOptions(
        port=9090,        # Changed to 9090
        grpc_port=50060,  # Changed to 50060
        additional_env_vars={
            "AUTHENTICATION_ANONYMOUS_ACCESS_ENABLED": "true"
        }
    ),
    skip_init_checks=True
)

# Delete collection if exists
try:
    client.collections.delete(collection_name)
except:
    pass

# Create collection
client.collections.create(
    name=collection_name,
    properties=[
        weaviate.properties.Property(name="text", data_type=weaviate.DataType.TEXT),
        weaviate.properties.Property(name="doc_id", data_type=weaviate.DataType.STRING),
        weaviate.properties.Property(name="source", data_type=weaviate.DataType.STRING)
    ]
)

# Create documents
dense_documents = [Document(page_content=text, metadata={"doc_id": str(i), "source": "dense"}) 
                  for i, text in enumerate(splits)]
sparse_documents = [Document(page_content=text, metadata={"doc_id": str(i), "source": "sparse"}) 
                   for i, text in enumerate(splits)]

# Initialize vectorstore
vectorstore = Weaviate(
    client=client,
    embedding=embedding_function,
    index_name=collection_name,
    text_key="text",
    attributes=["doc_id", "source"],
    by_text=False
)

# Batch import data
with client.batch.fixed_size(batch_size=100) as batch:
    for doc in tqdm(dense_documents, desc="Processing documents"):
        properties = {
            "text": doc.page_content,
            "doc_id": doc.metadata["doc_id"],
            "source": doc.metadata["source"]
        }
        vector = embedding_function.embed_query(doc.page_content)
        batch.add_object(
            collection_name=collection_name,
            properties=properties,
            vector=vector
        )

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)>

In [21]:
dense_retriever = vectorstore.as_retriever(search_kwargs={"k": 10})
sparse_retriever = BM25Retriever.from_documents(sparse_documents, k=10)
ensemble_retriever = EnsembleRetriever(retrievers=[dense_retriever, sparse_retriever], weights=[0.5, 0.5], c=0, k=10)


In [22]:
prompt = hub.pull("jclemens24/rag-prompt")


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [23]:
relevance_prompt_template = PromptTemplate.from_template(
    """
    Given the following question and retrieved context, determine if the context is relevant to the question.
    Provide a score from 1 to 5, where 1 is not at all relevant and 5 is highly relevant.
    Return ONLY the numeric score, without any additional text or explanation.

    Question: {question}
    Retrieved Context: {retrieved_context}

    Relevance Score:"""
)

In [24]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


In [25]:
def extract_score(llm_output):
    try:
        score = float(llm_output.strip())
        return score
    except ValueError:
        return 0

def conditional_answer(x):
    relevance_score = extract_score(x['relevance_score'])
    if relevance_score < 4:
        return "I don't know."
    else:
        return x['answer']

rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | RunnableParallel(
        {"relevance_score": (
            RunnablePassthrough()
            | (lambda x: relevance_prompt_template.format(question=x['question'], retrieved_context=x['context']))
            | llm
            | str_output_parser
        ), "answer": (
            RunnablePassthrough()
            | prompt
            | llm
            | str_output_parser
        )}
    )
    | RunnablePassthrough().assign(final_answer=conditional_answer)
)

In [26]:
rag_chain_with_source = RunnableParallel(
    {"context": ensemble_retriever, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)

In [27]:
result = rag_chain_with_source.invoke(user_query)
retrieved_docs = result['context']

print(f"Original Question: {user_query}\n")
print(f"Relevance Score: {result['answer']['relevance_score']}\n")
print(f"Final Answer:\n{result['answer']['final_answer']}\n\n")
print("Retrieved Documents:")
for i, doc in enumerate(retrieved_docs, start=1):
    # note: if using the Weaviate vectorstore, change 'id' to 'doc_id'
    print(f"Document {i}: Document ID: {doc.metadata['doc_id']} source: {doc.metadata['source']}")
    print(f"Content:\n{doc.page_content}\n")

Original Question: What are Google's environmental initiatives?

Relevance Score: 5

Final Answer:
Google's environmental initiatives focus on several key areas:

1. **Empowering Individuals**: Google aims to help 1 billion people make more sustainable choices through features in its products, such as eco-friendly routing in Google Maps, energy efficiency features in Google Nest thermostats, and carbon emissions information in Google Flights. The goal is to collectively reduce 1 gigaton of carbon equivalent emissions annually by 2030.

2. **Working Together**: Google collaborates with various partners and organizations, including the iMasons Climate Accord and the Nature Conservancy, to promote carbon reduction, support reforestation efforts, and enhance sustainability practices across industries.

3. **Operating Sustainably**: Google is committed to achieving net-zero carbon emissions, enhancing water stewardship, and pursuing a circular economy. This includes efforts to improve the e

KeyError: 'doc_id'